## Python Questionnaire
**Rose Alcolea**

In [30]:
import pandas as pd
import numpy as np
import os

#### Question 1
Can you sort a numerical list in Python?

In [7]:
lst = [5, 8, 1, 2, 10]

lst.sort()
print(lst)

[1, 2, 5, 8, 10]


#### Question 2
Write a code to count the number of capital letters in the “drivers_table.csv” file.


In [16]:
# Creating a loop that will count +1 everytime it encounters a capital letter
with open('./data/drivers_table.csv') as content:
    count = 0
    text = content.read()
    for letter in text:
        if letter.isupper():
            count += 1
print(count)

1384


#### Question 3
Write a function that lists the files in a path with a specific file extension.


In [15]:
for file in os.listdir("./data"):
    if file.endswith(".csv"):
        print(os.path.join("./data", file))

./data/orders_table.csv
./data/drivers_table.csv
./data/drivers_summary.csv


#### Question 4
Could you provide a code that executes the query you have created previously
in question 6 of SQL and export the result to a CSV?

Write a query showing the productivity of each driver per day. Productivity is the
number of orders that have been attempted per hour (orders attempted/hour)


In [19]:
# Opening the needed files with pandas
d = pd.read_csv('./data/drivers_table.csv')
o = pd.read_csv('./data/orders_table.csv')

In [20]:
d.head(1)

,id,driver,invoice_details,vehicle
0,1,ALBERTO,CARGO,Motorbike


In [21]:
o.head(1)

,id,Paack order number,driver_id,Deliver date,Delivery Start,Delivery End,Attempted time,Order Status,country,Company
0,13016706,64516863,99,01/10/2020,01/10/2020 11:00,01/10/2020 13:00,01/10/2020 11:53,delivered,ES,BIG STORE


In [29]:
# Merging the two tables
full = pd.merge(o, d, left_on = 'driver_id', right_on = 'id')
full.drop('id_y', axis=1, inplace=True)
full.head()

,id_x,Paack order number,driver_id,Deliver date,Delivery Start,Delivery End,Attempted time,Order Status,country,Company,driver,invoice_details,vehicle
0,13016706,64516863,99,01/10/2020,01/10/2020 11:00,01/10/2020 13:00,01/10/2020 11:53,delivered,ES,BIG STORE,LUIS MIGUEL,SAK,Van/Car
1,13017070,616544,99,01/10/2020,01/10/2020 11:00,01/10/2020 13:00,01/10/2020 11:59,delivered,ES,BIG STORE,LUIS MIGUEL,SAK,Van/Car
2,13026261,53283660,26,01/10/2020,01/10/2020 13:00,01/10/2020 15:00,01/10/2020 13:07,delivered,FR,LITTLE STORE,IKER,COSTA,Van/Car
3,13027827,75276804,26,01/10/2020,01/10/2020 10:00,01/10/2020 14:00,01/10/2020 11:44,delivered,ES,COFFE STORE,IKER,COSTA,Van/Car
4,13019173,4045773,26,01/10/2020,01/10/2020 10:00,01/10/2020 14:00,01/10/2020 13:13,delivered,ES,COFFE STORE,IKER,COSTA,Van/Car


In [51]:
# Creating a new column that takes only the hour at which each attempt was made
full['hour'] = [re.findall(' [0-9]{2}', x)[0] for x in full['Attempted time']]

In [77]:
# Creating a pivot table grouping by driver id and counting the amount of hours worked and the amount of attempts made
results = pd.pivot_table(full, values='hour', index='driver_id', aggfunc=['count', lambda x: len(x.unique())]).reset_index()
results.columns = ['Driver Id', 'Attempts Made', 'Hours Worked']

# Adding a column that calculates each driver's productivity
results['Productivity'] = results['Attempts Made']/results['Hours Worked']
results.sort_values('Productivity', ascending=False)

,Driver Id,Attempts Made,Hours Worked,Productivity
7,12,4,1,4.0
30,35,3,1,3.0
38,43,3,1,3.0
83,93,3,1,3.0
50,56,3,1,3.0
...,...,...,...,...
52,58,2,2,1.0
54,60,1,1,1.0
55,62,2,2,1.0
56,63,1,1,1.0


#### Question 5
Can you write a code that executes a query in one database and insert the data
in a different database?


In [ ]:
# Defining the connexion to each database
conn_p = p.connect("dbname='paack_1' user='postgres' password='postgres' host='localhost'")
strt="dbname='paack_2' user='postgres' password='postgres' host='localhost'"
cur_p = conn_p.cursor();

# Writing the query I want to insert in the other database & executing the transfer
# Note: I use dblink for faster processing
cur_p.execute("select dblink_connect('tmpdb',%s);",(strt,));
cur_p.execute("""
  insert into "new_table" (driver_id, country, Company)
  SELECT * FROM dblink('tmpdb')
""")

# Disconnecting
cur_p.execute("SELECT dblink_disconnect('tmpdb');");

# Note: I was not able to check if this code runs properly but it should. 
# In a real case scenario I would check it and modify it until it achieves the desired result.